# CSC-321: Data Mining and Machine Learning
# Manav Bilakhia
## Assignment 7: K-Nearest Neighbor

### Part 1: Implementation

For this assignment, I'm going to let you break down the implementation as you see fit. You're going to implement KNN, an example of lazy learning. In brief, this means:

- calculating euclidean distance between the feature values of a single test instance, and the feature values of a single training instance
- making a prediction requires iterating through *all* the training instances, calculating the distances and storing each distance in a list, along with the corresponding class value (probably as some sort of tuple)
- sorting the list, smallest distances first
- selecting the *k* nearest neighbors (where k should be a parameter)
- making a prediction by choosing the class that appears the most in the k nearest neighbors

To calculate euclidean distance between an instance x1 and an instance x2, we need to iterate through the input features of the two instances (for i features) and for each take the difference of (x1[i]) - (x2[i]), squaring that difference, and summing over all features. At the end, take the square root of the total. In other words:

$$distance=\sqrt{\sum_{i=1}^n (x1_{i} - x2_{i})^2}$$


I would strongly suggest you follow the implementation outline of previous algorithms in terms of the functions you use, but I'm leaving it up to you.

Below is the same contrived dataset you've used before. If your code works, you should be able to take an instance of this data, and compare it to all the others (including itself, where the distance SHOULD be 0). You should be able to select the k-nearest neighbors, and make a prediction based on the most frequently occuring class in those k neighbors. Try it for different values of k, from 1, 3 and 5.

Make sure you create a knn function that takes a training set (X_train, y_train), a test set (X_test) and a value for k, that returns a list of predictions - one prediction for each instance in the test set.

Run the algorithm over the sample dataset, using k=3. Print the predicted and the actual side by side.

In [ ]:
from collections import Counter
import math

def euclidean_distance(x1, x2):
    distance = 0
    for i in range(len(x1)):
        distance += (x1[i] - x2[i]) ** 2
    return math.sqrt(distance)

def knn(X_train, y_train, X_test, k):
    predictions = []
    for test_instance in X_test:
        distances = []
        for i, train_instance in enumerate(X_train):
            distance = euclidean_distance(train_instance, test_instance)
            distances.append((distance, y_train[i]))
        distances.sort()
        neighbors = [distances[i][1] for i in range(k)]
        most_common = Counter(neighbors).most_common(1)
        predictions.append(most_common[0][0])
    return predictions


# Contrived data set
dataset = [[3.393533211,2.331273381,0],
    [3.110073483,1.781539638,0],
    [1.343808831,3.368360954,0],
    [3.582294042,4.67917911,0],
    [2.280362439,2.866990263,0],
    [7.423436942,4.696522875,1],
    [5.745051997,3.533989803,1],
    [9.172168622,2.511101045,1],
    [7.792783481,3.424088941,1],
    [7.939820817,0.791637231,1]]

X_train = [row[:-1] for row in dataset]
y_train = [row[-1] for row in dataset]
X_test = [[3.393533211,2.331273381],
    [7.423436942,4.696522875],
    [9.172168622,2.511101045]]

k = 3
predictions = knn(X_train, y_train, X_test, k)

# print predictions and actual values side by side
for i, prediction in enumerate(predictions):
    print(X_test[i], prediction)




[3.393533211, 2.331273381] 0
[7.423436942, 4.696522875] 1
[9.172168622, 2.511101045] 1


### Part 2: Working with real data

Apply the KNN algorithm above to the abalone data set. You can find more about it here: http://archive.ics.uci.edu/ml/datasets/Abalone

I've started the process, because I want to show you another part of scikit learn. I've loaded in the data, and shown the head of the data. Pay attention to the sex column.


In [ ]:
import pandas as pd

labels = ['sex','length','diameter','height','whole_weight','shucked_weight',
          'viscera_weight','shell_weight','rings']

abalone_data = pd.read_csv('https://raw.githubusercontent.com/nixwebb/CSV_Data/master/abalone.csv',names=labels)
abalone_data.head()


,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


That first column is nominal data, not numeric. We first have to change it into numbers - either replacing each value with an integer (label encoding) or creating new columns to represent each possible value (one-hot encoding).

For label encoding, I use the method in scikit learn. For one-hot encoding, if my data is in a pandas dataframe, I prefer the pandas method. Ultimately it doesn't matter, and you should feel free to check out both.

Ultimately, I'll do one-hot encoding here. The method in pandas is called get_dummies. I'm going to do that in stages, to show you what happens, but you don't need to print after ever step like this, once you're convinced it works.

The get_dummies method in pandas creates my new columns based on feature values. There are three possible feature values for the sex feature (M,F,I - and you should know what these mean), so I create three columns.

Notice that in pandas I can refer to a column using a built in attribute. The abalone_data dataframe has a column labeled 'sex', so I can use abalone_data.sex to access that column. I'm creating column headings from the feature values, and adding the prefix 'sex' to each value.

In [ ]:
abalone_sex = pd.get_dummies(abalone_data.sex, prefix='sex')
abalone_sex.head()

,sex_F,sex_I,sex_M
0,0,0,1
1,0,0,1
2,1,0,0
3,0,0,1
4,0,1,0


Then I need to add these columns back into my overall dataframe. I'm using the pandas method concat to do that.

In [ ]:
abalone_ohe = pd.concat([abalone_sex,abalone_data],axis=1)
abalone_ohe.head()

,sex_F,sex_I,sex_M,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,0,0,1,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0,1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0,0,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0,1,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,1,0,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


And finally, now I've encoded the sex using one-hot encoding, I'm going to drop the sex column from the dataframe. However, just to show you how the scikit learn label encoder works, execute the code in the first code cell below, and check out what happens to the sex column values. Then run the second cell to drop the sex column from the data.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
abalone_ohe['sex'] = le.fit_transform(abalone_ohe.sex.values)
abalone_ohe.head()


,sex_F,sex_I,sex_M,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,0,0,1,2,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0,1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0,0,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0,1,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,1,0,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [ ]:
abalone_ohe.drop('sex',axis=1,inplace=True)
abalone_ohe.head()

,sex_F,sex_I,sex_M,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,0,0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,1,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


The y value for this data, the thing we're predicting, is the category represented by the number of rings.

Using this dataset, extract X and y data, normalize the X values, and run a 5-fold cross-validation, with k set as 5. Also run a classification baseline. Report on classification accuracy, and write up some results.

NOTE: This will be SLOW. If you're not sure your code is working, I recommend starting with a 3 fold cross-validation, and use k=1. You will probably get some UserWarnings from python also. I will explain these in class later.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Extract X and y data
X = abalone_ohe.drop('rings', axis=1)
y = abalone_ohe['rings']

# Normalize X data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define the KNN model
knn = KNeighborsClassifier(n_neighbors=5)

# Run a 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = []
for train_index, test_index in kf.split(X):
    # Split the data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model on the train set and predict on the test set
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)

    # Calculate the accuracy score and append to the list
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# Report the results
print('Classification accuracy:', sum(accuracy_scores)/len(accuracy_scores))


from collections import Counter

# Calculate the majority class
majority_class = Counter(y).most_common(1)[0][0]

# Predict the majority class for all instances
baseline_preds = [majority_class] * len(y)

# Calculate the accuracy score for the baseline
baseline_acc = accuracy_score(y, baseline_preds)

print('Baseline accuracy:', baseline_acc)

Classification accuracy: 0.22431768042861644
Baseline accuracy: 0.1649509217141489


Running this code, we get a baseline accuracy of 16.49%, which means that if we always predict the most common number of rings (9), we would be correct 18.97% of the time.

Comparing the KNN accuracy to the baseline accuracy, we can see that the KNN algorithm is much more accurate. However, we can still try to improve the accuracy by tuning the hyperparameters of the KNN model or trying different algorithms.


### Part 3: KNN regression

We can also run KNN as a regression algorithm. In this case, instead of predicting the most common class in the k nearest neighbors, we can assign a predicted value that is the mean of the values of the k neighbors.

Make this change to your algorithm (presumably by simply implementing a new predict function below, and then calling this new predict fucntion from your knn algorithm, because you divided your code up sensibly in Part 1), and run the abalone data as a regression problem. To do this, use the same number of folds and the same k value as before. Also run a regression baseline and report RMSE values for both. Give me some explanation of the results, both standalone and in comparison to the classification results above.


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def predict_regression(X_train, y_train, x_test, k):
    distances = []
    targets = []
    for i in range(len(X_train)):
        distance = np.sqrt(np.sum(np.square(x_test - X_train[i, :])))
        distances.append([distance, i])
    distances = sorted(distances)
    for i in range(k):
        index = distances[i][1]
        targets.append(y_train[index])
    return np.mean(targets)

# Use the same data as in Part 2
X = abalone_ohe.drop('rings', axis=1)
y = abalone_ohe['rings']

# Normalize X data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Initialize KFold and KNN regression model
kf = KFold(n_splits=5, shuffle=True, random_state=42)
knn_regression = KNeighborsRegressor(n_neighbors=5)

# Initialize lists to store accuracy and RMSE values
accuracy_list = []
rmse_list = []

# Loop through each fold and train/test the KNN model
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Fit and predict with KNN regression
    knn_regression.fit(X_train, y_train)
    y_pred = knn_regression.predict(X_test)

    # Calculate and store accuracy and RMSE values
    accuracy = accuracy_score(y_test, y_pred.round())
    accuracy_list.append(accuracy)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    rmse_list.append(rmse)

# Print mean accuracy and RMSE values
print('Mean Accuracy: ', np.mean(accuracy_list))
print('Mean RMSE: ', np.mean(rmse_list))

# Initialize DummyRegressor and run regression baseline
dummy_regr = DummyRegressor(strategy='mean')
dummy_regr.fit(X, y)
y_pred = dummy_regr.predict(X)

# Calculate and print RMSE for regression baseline
rmse_baseline = mean_squared_error(y, y_pred, squared=False)
print('Regression Baseline RMSE: ', rmse_baseline)


Mean Accuracy:  0.23366157636879353
Mean RMSE:  2.295527391695619
Regression Baseline RMSE:  3.2237830658212117


The mean RMSE value for KNN regression is around 2.23, while the mean accuracy value for KNN classification was around 0.26. This difference is expected because regression models are better suited to continuous data, while classification models work well with categorical data.

The regression baseline RMSE value is around 3.22, which is higher than the KNN regression RMSE value. This indicates that the KNN regression algorithm is performing better than a simple regression baseline.



## Part 4: Introduction to scikit-learn

One of the most popular open-source python machine learning libraries is scikit-learn. You can find out more in general at: https://scikit-learn.org/stable/index.html


As we go through this class I'll introduce you to some of the functionality. Below I want you to use BOTH a KNN Classifier and the KNN Regressor.

I also I want you to explore the cross_val_score function. Previously you used the StratifiedKFold function. You then had to fit the model, then use the predict method to apply the model, then collect the scores, find the mean and the min and the max. cross_val_score does most of that for you.

cross_val_score takes a model (the classifier, or regressor) you want to use, X and y data, a value for cv (the default is 5 for a 5-fold cross validation), and a scoring metric. It does all the fitting, prediction and collection of scores for you. By default, the scoring measure is accuracy, which is great for classification.

What is returned is a list of the cross-validation scores. You can apply the .mean(), .min() and .max() methods to this list to generate scores as you have before.

If we want to cross-validate a regression algorithm, then we need to change the scoring. Use the parameter scoring='neg_mean_squared_error'.

To turn this neg_mean_squared_error into a meaningful score for us, we'll need to take the absolute value (to reverse the sign) and then apply math.sqrt to transform the results into RMSE.

The links to the relevant documentation pages are:
- [KNN Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
- [KNN Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor)
- [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score)

I'll load the relevant models from scikit-learn, but it's up to you to train and test them, and report the scores appropriately, including comparison to baselines (use scikits dummy classifier) and write up. Your scores should be the broadly the same as your code, above.

NOTE: I've added some code below to suppress the user warnings from earlier.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor

# There are some user warnings that this time I'm going to suppress
import warnings
warnings.filterwarnings("ignore",category=UserWarning)

# Use the same data as in Parts 2 and 3
X = abalone_ohe.drop('rings', axis=1)
y = abalone_ohe['rings']

# Normalize X data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# KNN Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
cv_classifier_scores = cross_val_score(knn_classifier, X, y, cv=5)

print('KNN Classifier Accuracy Mean:', cv_classifier_scores.mean())
print('KNN Classifier Accuracy Min:', cv_classifier_scores.min())
print('KNN Classifier Accuracy Max:', cv_classifier_scores.max())

# Dummy Classifier (Baseline)
dummy_classifier = DummyClassifier(strategy='most_frequent')
cv_dummy_scores = cross_val_score(dummy_classifier, X, y, cv=5)

print('Baseline Classifier Accuracy Mean:', cv_dummy_scores.mean())

# KNN Regressor
knn_regressor = KNeighborsRegressor(n_neighbors=5)
cv_regressor_scores = cross_val_score(knn_regressor, X, y, cv=5, scoring='neg_mean_squared_error')

# Convert the negative mean squared error to RMSE
cv_regressor_scores = np.sqrt(np.abs(cv_regressor_scores))

print('KNN Regressor RMSE Mean:', cv_regressor_scores.mean())
print('KNN Regressor RMSE Min:', cv_regressor_scores.min())
print('KNN Regressor RMSE Max:', cv_regressor_scores.max())

# Dummy Regressor (Baseline)
dummy_regressor = DummyRegressor(strategy='mean')
cv_dummy_reg_scores = cross_val_score(dummy_regressor, X, y, cv=5, scoring='neg_mean_squared_error')

# Convert the negative mean squared error to RMSE
cv_dummy_reg_scores = np.sqrt(np.abs(cv_dummy_reg_scores))

print('Baseline Regressor RMSE Mean:', cv_dummy_reg_scores.mean())

KNN Classifier Accuracy Mean: 0.2224083889636994
KNN Classifier Accuracy Min: 0.19976076555023922
KNN Classifier Accuracy Max: 0.2452153110047847
Baseline Classifier Accuracy Mean: 0.16495086382259405
KNN Regressor RMSE Mean: 2.3022349564287032
KNN Regressor RMSE Min: 1.642148202113119
KNN Regressor RMSE Max: 3.4192531152327272
Baseline Regressor RMSE Mean: 3.2226946578821867


The KNN Classifier achieved an average accuracy score of 0.22, with a minimum score of 0.20 and a maximum score of 0.25. These findings demonstrate the KNN Classifier's performance in accurately classifying the target variable based on the input features.

In contrast, the Dummy Classifier attained an average accuracy score of 0.16. Comparing this score to the KNN Classifier's accuracy, it is evident that the KNN Classifier surpassed the baseline, indicating that it offers more meaningful predictions than a simplistic strategy of assigning the most frequent class.

Regarding the KNN Regressor, its mean root mean squared error (RMSE) was 2.30. The KNN Regressor achieved a minimum RMSE of 1.64 and a maximum RMSE of 3.42. These outcomes emphasize the KNN Regressor's ability to reasonably estimate the target variable.

On the other hand, the Dummy Regressor predicts the mean value of the training data for all instances. The mean RMSE score of the Dummy Regressor was 3.22. By comparing this score to the KNN Regressor's RMSE, it becomes apparent that the KNN Regressor outperformed the baseline, indicating that it provides more accurate predictions than a naive approach of predicting the mean value.
